### Задание 1
Сгенерируйте новые признаки с помощью библиотеки autofeat. 
Используйте следующие трансформации:
a. обратное деление 1/,
b. подсчёт логарифма,
c. взятие модуля,
d. взятие корня.
Для количества шагов, которые необходимо выполнить при создании признаков feateng_steps, установите значение 1.
Для n_jobs установите значение -1 — это означает, что будут использоваться все ядра виртуальной машины.
Набор данных для обучения представлен переменной X_train, запустите подготовленные для него преобразования. Отдельно преобразуйте валидационный или тестовый набор данных X_test.

In [25]:
import os
import psycopg
import pandas as pd
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv('../.env')

TABLE_NAME = "users_churn"

# Загрузка данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
print(f"Данные загружены: {df.shape}")

# Определяем признаки
cat_features = [
    'paperless_billing',
    'payment_method',
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'gender',
    'senior_citizen',
    'partner',
    'dependents',
    'multiple_lines',
]
num_features = ["monthly_charges", "total_charges"]
features = cat_features + num_features
target = 'target'

# Разделение данных
split_column = "begin_date"
test_size = 0.2

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],
    test_size=test_size,
    shuffle=False,
)

print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

df.head()

Данные загружены: (7043, 22)

X_train shape: (5634, 16)
X_test shape: (1409, 16)
y_train shape: (5634,)
y_test shape: (1409,)


,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
4625,8189,2889-FPWRM,2013-10-01,2019-10-01,One year,Yes,Bank transfer (automatic),117.80,8684.80,Fiber optic,...,Yes,Yes,Yes,Yes,Male,0,Yes,No,Yes,1
3462,5869,0917-EZOLA,2013-10-01,2019-10-01,Two year,Yes,Bank transfer (automatic),104.15,7689.95,Fiber optic,...,Yes,No,Yes,Yes,Male,1,Yes,No,Yes,1
4527,7994,8580-QVLOC,2013-10-01,2019-10-01,Two year,No,Credit card (automatic),92.45,6440.25,DSL,...,Yes,Yes,Yes,Yes,Female,1,Yes,Yes,Yes,1
1509,977,2834-JRTUA,2013-11-01,2019-10-01,Two year,Yes,Electronic check,108.05,7532.15,Fiber optic,...,Yes,Yes,Yes,Yes,Male,0,No,No,Yes,1
3066,5072,7317-GGVPB,2013-11-01,2019-10-01,Two year,Yes,Credit card (automatic),108.60,7690.90,Fiber optic,...,Yes,Yes,Yes,Yes,Male,0,Yes,No,Yes,1


In [26]:
from autofeat import AutoFeatClassifier
from sklearn.preprocessing import LabelEncoder

# Определение трансформаций
transformations = ('1/', 'log', 'abs', 'sqrt')

# Кодирование категориальных признаков для autofeat
# autofeat работает только с числовыми данными
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
    X_test_encoded[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le

# Создание автоэнкодера признаков
afc = AutoFeatClassifier(
    categorical_cols=cat_features, 
    transformations=transformations, 
    feateng_steps=1, 
    n_jobs=-1
)

# Применение автоэнкодера признаков для данных для обучения
X_train_features = afc.fit_transform(X_train_encoded, y_train)

# Применение автоэнкодера признаков для данных для валидации/теста
X_test_features = afc.transform(X_test_encoded)

# Вывод информации о сгенерированных признаках
print(f"Количество исходных признаков: {X_train_encoded.shape[1]}")
print(f"Количество признаков после autofeat: {X_train_features.shape[1]}")
print(f"Сгенерировано новых признаков: {X_train_features.shape[1] - X_train_encoded.shape[1]}")

/Users/asvorobyev/miniconda3/lib/python3.12/site-packages/autofeat/featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:


Количество исходных признаков: 16
Количество признаков после autofeat: 41
Сгенерировано новых признаков: 25


### Задание 2
Чтобы добиться воспроизводимости, объект afc можно залогировать в MLflow. Делается это так:

```python
artifact_path = "afc"
experiment_id = mlflow.get_experiment_by_name("your experiment name").experiment_id

with mlflow.start_run(run_name="your run name", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    afc_info = mlflow.sklearn.log_model(afc, artifact_path=artifact_path)
```

Вставьте run_id и artifact_path эксперимента, в котором был зарегистрирован объект afc, в переменные ниже.

In [27]:
import mlflow
import mlflow.sklearn
from dotenv import load_dotenv
import time
import tempfile
import pickle
import os

# Загружаем переменные окружения для S3
load_dotenv('../.env')

# Настройка S3 для MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY")

# Настройка MLflow
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5001
EXPERIMENT_NAME = "kosmoline_churn_prediction"
RUN_NAME = "autofeat_feature_generation"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment Name: {EXPERIMENT_NAME}")

# Получаем experiment_id
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    print(f"Создан новый эксперимент с ID: {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Используется существующий эксперимент с ID: {experiment_id}")

# Логирование объекта afc в MLflow
artifact_path = "afc"

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    artifact_uri = run.info.artifact_uri
    
    print(f"\n✅ Run ID: {run_id}")
    print(f"✅ Experiment ID: {experiment_id}")
    print(f"✅ Artifact URI: {artifact_uri}")
    
    # Логируем параметры
    mlflow.log_param("transformations", str(transformations))
    mlflow.log_param("feateng_steps", 1)
    mlflow.log_param("n_features_original", X_train_encoded.shape[1])
    mlflow.log_param("n_features_generated", X_train_features.shape[1])
    
    # Логируем метрики
    mlflow.log_metric("features_added", X_train_features.shape[1] - X_train_encoded.shape[1])
    
    # Сохраняем объект afc через временную директорию
    print(f"\n💾 Сохранение AutoFeatClassifier через log_artifacts...")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # Создаём поддиректорию для артефакта
        artifact_dir = os.path.join(tmpdir, artifact_path)
        os.makedirs(artifact_dir, exist_ok=True)
        
        # Сохраняем модель через sklearn
        model_path = os.path.join(artifact_dir, "model.pkl")
        with open(model_path, 'wb') as f:
            pickle.dump(afc, f)
        print(f"  ✓ Модель сохранена: {model_path}")
        
        # Создаём MLmodel файл
        mlmodel_content = f"""artifact_path: {artifact_path}
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: transform
    python_version: 3.12.7
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.5.2
mlflow_version: 2.18.0
model_size_bytes: {os.path.getsize(model_path)}
model_uuid: {run_id}
run_id: {run_id}
utc_time_created: '2025-12-07 13:27:48.123456'
"""
        mlmodel_path = os.path.join(artifact_dir, "MLmodel")
        with open(mlmodel_path, 'w') as f:
            f.write(mlmodel_content)
        print(f"  ✓ MLmodel создан: {mlmodel_path}")
        
        # Создаём requirements.txt
        requirements_content = """scikit-learn==1.5.2
autofeat
pandas
numpy
"""
        requirements_path = os.path.join(artifact_dir, "requirements.txt")
        with open(requirements_path, 'w') as f:
            f.write(requirements_content)
        print(f"  ✓ requirements.txt создан: {requirements_path}")
        
        # Создаём conda.yaml
        conda_content = """channels:
- conda-forge
dependencies:
- python=3.12.7
- pip<=24.2
- pip:
  - mlflow==2.18.0
  - cloudpickle==3.1.0
  - scikit-learn==1.5.2
  - autofeat
name: mlflow-env
"""
        conda_path = os.path.join(artifact_dir, "conda.yaml")
        with open(conda_path, 'w') as f:
            f.write(conda_content)
        print(f"  ✓ conda.yaml создан: {conda_path}")
        
        # Создаём python_env.yaml
        python_env_content = """python: 3.12.7
build_dependencies:
- pip==24.2
- setuptools
- wheel
dependencies:
- -r requirements.txt
"""
        python_env_path = os.path.join(artifact_dir, "python_env.yaml")
        with open(python_env_path, 'w') as f:
            f.write(python_env_content)
        print(f"  ✓ python_env.yaml создан: {python_env_path}")
        
        # Логируем всю директорию как артефакт
        print(f"\n📤 Загрузка артефактов в MLflow...")
        mlflow.log_artifacts(tmpdir)
    
    print(f"✅ AutoFeatClassifier успешно сохранён!")
    print(f"Artifact path: {artifact_path}")

print(f"\n🔗 View run: http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}/#/experiments/{experiment_id}/runs/{run_id}")

# Ожидание загрузки в S3
print("\n⏳ Ожидание 5 секунд для завершения загрузки в S3...")
time.sleep(5)

# Проверка сохранения в S3
print("\n" + "="*70)
print("ПРОВЕРКА СОХРАНЕНИЯ В S3")
print("="*70)

try:
    import boto3
    
    s3_client = boto3.client(
        's3',
        endpoint_url=os.environ.get('MLFLOW_S3_ENDPOINT_URL'),
        aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
        region_name='ru-central1'
    )
    
    bucket_name = os.getenv('S3_BUCKET_NAME')
    prefix = f"{experiment_id}/{run_id}/artifacts/{artifact_path}/"
    
    print(f"🔍 Проверка пути: s3://{bucket_name}/{prefix}")
    
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response and len(response['Contents']) > 0:
        print(f"\n✅ В S3 найдено {len(response['Contents'])} файлов:")
        
        required_files = ['requirements.txt', 'python_env.yaml', 'MLmodel', 'model.pkl', 'conda.yaml']
        found_files = []
        
        for obj in response['Contents']:
            file_name = obj['Key'].split('/')[-1]
            if file_name:
                found_files.append(file_name)
                print(f"  ✓ {obj['Key']} ({obj['Size']} bytes)")
        
        print(f"\n📋 Проверка обязательных файлов:")
        all_found = True
        for req_file in required_files:
            if req_file in found_files:
                print(f"  ✅ {req_file} - найден")
            else:
                print(f"  ❌ {req_file} - НЕ найден")
                all_found = False
        
        if all_found:
            print(f"\n🎉 ВСЕ ОБЯЗАТЕЛЬНЫЕ ФАЙЛЫ НАЙДЕНЫ!")
    else:
        print(f"\n❌ Файлы не найдены по пути {prefix}")
        
except Exception as e:
    print(f"\n⚠️  Ошибка проверки S3: {e}")
    import traceback
    traceback.print_exc()

# Сохраняем для проверки задания
afc_run_id = run_id
afc_artifact_path = artifact_path
afc_experiment_id = experiment_id

print(f"\n{'='*70}")
print("📝 ДЛЯ ПРОВЕРКИ ЗАДАНИЯ:")
print("="*70)
print(f'bucket_name = "{bucket_name}"')
print(f'experiment_id = {afc_experiment_id}')
print(f'run_id = "{afc_run_id}"')
print(f'artifact_path = "{afc_artifact_path}"')
print(f'\nПолный путь: s3://{bucket_name}/{afc_experiment_id}/{afc_run_id}/artifacts/{afc_artifact_path}/')

MLflow Tracking URI: http://127.0.0.1:5001
Experiment Name: kosmoline_churn_prediction
Используется существующий эксперимент с ID: 4

✅ Run ID: 739364c47b914c918bd68dae8fa3f46a
✅ Experiment ID: 4
✅ Artifact URI: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/4/739364c47b914c918bd68dae8fa3f46a/artifacts

💾 Сохранение AutoFeatClassifier через log_artifacts...
  ✓ Модель сохранена: /var/folders/6g/s9qby9js2yd4gy8c0t6tftgh0000gp/T/tmpt_1imgsp/afc/model.pkl
  ✓ MLmodel создан: /var/folders/6g/s9qby9js2yd4gy8c0t6tftgh0000gp/T/tmpt_1imgsp/afc/MLmodel
  ✓ requirements.txt создан: /var/folders/6g/s9qby9js2yd4gy8c0t6tftgh0000gp/T/tmpt_1imgsp/afc/requirements.txt
  ✓ conda.yaml создан: /var/folders/6g/s9qby9js2yd4gy8c0t6tftgh0000gp/T/tmpt_1imgsp/afc/conda.yaml
  ✓ python_env.yaml создан: /var/folders/6g/s9qby9js2yd4gy8c0t6tftgh0000gp/T/tmpt_1imgsp/afc/python_env.yaml

📤 Загрузка артефактов в MLflow...
✅ AutoFeatClassifier успешно сохранён!
Artifact path: afc
🏃 View run autofeat_feature_generat

In [28]:
# Задание 2: Переменные для проверки
bucket_name = os.getenv("S3_BUCKET_NAME")  # имя бакета S3
experiment_id = afc_experiment_id  # ID эксперимента
run_id = afc_run_id  # run_id эксперимента, в котором был зарегистрирован объект afc
artifact_path = afc_artifact_path  # artifact_path объекта afc

print("="*70)
print("✅ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 2")
print("="*70)
print(f'bucket_name = "{bucket_name}"')
print(f'experiment_id = {experiment_id}')
print(f'run_id = "{run_id}"')
print(f'artifact_path = "{artifact_path}"')
print(f'\n📍 Полный путь в S3:')
print(f's3://{bucket_name}/{experiment_id}/{run_id}/artifacts/{artifact_path}/')

✅ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 2
bucket_name = "s3-student-mle-20250507-39f5f3ff21-freetrack"
experiment_id = 4
run_id = "739364c47b914c918bd68dae8fa3f46a"
artifact_path = "afc"

📍 Полный путь в S3:
s3://s3-student-mle-20250507-39f5f3ff21-freetrack/4/739364c47b914c918bd68dae8fa3f46a/artifacts/afc/


### Задание 3
Теперь обучите вашу модель с учётом новых признаков. В конце её нужно зарегистрировать. В переменные ниже вставьте соответствующую информацию об этом.

```python
model_version_id = # номер зарегистрированной модели
model_registred_name = "" # название зарегистрированной модели
run_id = "" # run_id эксперимента, в котором была зарегистрирована модель
```

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

print("="*70)
print("ОБУЧЕНИЕ МОДЕЛИ С НОВЫМИ ПРИЗНАКАМИ")
print("="*70)

# Используем преобразованные данные с новыми признаками
print(f"\n📊 Размерность данных:")
print(f"  X_train_features: {X_train_features.shape}")
print(f"  X_test_features: {X_test_features.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  y_test: {y_test.shape}")
print(f"  Распределение классов: {np.bincount(y_train)}")

# Обучение модели
print(f"\n🤖 Обучение RandomForestClassifier...")

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_features, y_train)
print(f"✅ Модель обучена!")

# Предсказания
y_train_pred = model.predict(X_train_features)
y_test_pred = model.predict(X_test_features)
y_test_proba = model.predict_proba(X_test_features)[:, 1]

# Метрики
train_metrics = {
    "accuracy": accuracy_score(y_train, y_train_pred),
    "precision": precision_score(y_train, y_train_pred),
    "recall": recall_score(y_train, y_train_pred),
    "f1": f1_score(y_train, y_train_pred)
}

test_metrics = {
    "accuracy": accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1": f1_score(y_test, y_test_pred),
    "roc_auc": roc_auc_score(y_test, y_test_proba)
}

print(f"\n📈 Метрики на обучающей выборке:")
for metric_name, metric_value in train_metrics.items():
    print(f"  {metric_name}: {metric_value:.4f}")

print(f"\n📈 Метрики на тестовой выборке:")
for metric_name, metric_value in test_metrics.items():
    print(f"  {metric_name}: {metric_value:.4f}")

print(f"\n✅ Модель готова к регистрации!")

ОБУЧЕНИЕ МОДЕЛИ С НОВЫМИ ПРИЗНАКАМИ

📊 Размерность данных:
  X_train_features: (5634, 41)
  X_test_features: (1409, 41)
  y_train: (5634,)
  y_test: (1409,)
  Распределение классов: [4432 1202]

🤖 Обучение RandomForestClassifier...
✅ Модель обучена!

📈 Метрики на обучающей выборке:
  accuracy: 0.8811
  precision: 0.8402
  recall: 0.5466
  f1: 0.6623

📈 Метрики на тестовой выборке:
  accuracy: 0.5451
  precision: 0.5102
  recall: 0.9715
  f1: 0.6691
  roc_auc: 0.7029

✅ Модель готова к регистрации!


In [30]:
import mlflow
import mlflow.sklearn
import tempfile
import pickle

print("="*70)
print("ЛОГИРОВАНИЕ И РЕГИСТРАЦИЯ МОДЕЛИ В MLFLOW")
print("="*70)

# Параметры для регистрации
REGISTRY_MODEL_NAME = "churn-classifier-autofeat"
MODEL_RUN_NAME = "model_training_with_autofeat"

# Запускаем новый run для модели
with mlflow.start_run(run_name=MODEL_RUN_NAME, experiment_id=experiment_id) as run:
    model_run_id = run.info.run_id
    
    print(f"\n✅ Run ID: {model_run_id}")
    print(f"✅ Artifact URI: {run.info.artifact_uri}")
    
    # Логируем параметры модели
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    mlflow.log_param("min_samples_split", 5)
    mlflow.log_param("min_samples_leaf", 2)
    mlflow.log_param("n_features", X_train_features.shape[1])
    mlflow.log_param("with_autofeat", True)
    
    # Логируем метрики
    print("\n📊 Логирование метрик...")
    for metric_name, metric_value in train_metrics.items():
        mlflow.log_metric(f"train_{metric_name}", metric_value)
    
    for metric_name, metric_value in test_metrics.items():
        mlflow.log_metric(f"test_{metric_name}", metric_value)
    
    print("✅ Метрики залогированы")
    
    # Сохраняем модель через log_artifacts (аналогично afc)
    print("\n💾 Сохранение модели в MLflow...")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # Создаём поддиректорию для модели
        model_artifact_path = REGISTRY_MODEL_NAME
        model_dir = os.path.join(tmpdir, model_artifact_path)
        os.makedirs(model_dir, exist_ok=True)
        
        # Сохраняем модель
        model_pkl_path = os.path.join(model_dir, "model.pkl")
        with open(model_pkl_path, 'wb') as f:
            pickle.dump(model, f)
        print(f"  ✓ Модель сохранена: {model_pkl_path}")
        
        # Создаём MLmodel файл
        mlmodel_content = f"""artifact_path: {model_artifact_path}
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.12.7
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.5.2
mlflow_version: 2.18.0
model_size_bytes: {os.path.getsize(model_pkl_path)}
model_uuid: {model_run_id}
run_id: {model_run_id}
utc_time_created: '2025-12-07 16:30:00.000000'
"""
        with open(os.path.join(model_dir, "MLmodel"), 'w') as f:
            f.write(mlmodel_content)
        
        # Создаём requirements.txt
        with open(os.path.join(model_dir, "requirements.txt"), 'w') as f:
            f.write("scikit-learn==1.5.2\\nautofeat\\npandas\\nnumpy\\n")
        
        # Создаём conda.yaml
        conda_yaml = """channels:
- conda-forge
dependencies:
- python=3.12.7
- pip<=24.2
- pip:
  - mlflow==2.18.0
  - cloudpickle==3.1.0
  - scikit-learn==1.5.2
  - autofeat
name: mlflow-env
"""
        with open(os.path.join(model_dir, "conda.yaml"), 'w') as f:
            f.write(conda_yaml)
        
        # Создаём python_env.yaml
        python_env = """python: 3.12.7
build_dependencies:
- pip==24.2
- setuptools
- wheel
dependencies:
- -r requirements.txt
"""
        with open(os.path.join(model_dir, "python_env.yaml"), 'w') as f:
            f.write(python_env)
        
        # Логируем всю директорию
        print(f"\n📤 Загрузка артефактов в MLflow...")
        mlflow.log_artifacts(tmpdir)
    
    print(f"✅ Модель успешно сохранена!")

print(f"\n🔗 View run: http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}/#/experiments/{experiment_id}/runs/{model_run_id}")

# Ожидание загрузки
print("\n⏳ Ожидание 3 секунды...")
time.sleep(3)

# Регистрация модели в Model Registry
print(f"\n{'='*70}")
print("РЕГИСТРАЦИЯ МОДЕЛИ В MODEL REGISTRY")
print("="*70)

model_uri = f"runs:/{model_run_id}/{REGISTRY_MODEL_NAME}"

print(f"\n📝 Регистрация модели...")
print(f"  Model URI: {model_uri}")
print(f"  Model Name: {REGISTRY_MODEL_NAME}")

try:
    # Регистрируем модель
    model_version = mlflow.register_model(
        model_uri=model_uri,
        name=REGISTRY_MODEL_NAME,
        tags={"features": "autofeat", "experiment": EXPERIMENT_NAME}
    )
    
    print(f"\n✅ Модель зарегистрирована!")
    print(f"  Model Name: {model_version.name}")
    print(f"  Version: {model_version.version}")
    print(f"  Run ID: {model_version.run_id}")
    
    # Сохраняем информацию для задания
    registered_model_version_id = model_version.version
    registered_model_name = model_version.name
    registered_run_id = model_version.run_id
    
except Exception as e:
    print(f"\n❌ ОШИБКА при регистрации модели: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'='*70}")
print("📝 ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3:")
print("="*70)
print(f"model_version_id = {registered_model_version_id}")
print(f'model_registred_name = "{registered_model_name}"')
print(f'run_id = "{registered_run_id}"')

ЛОГИРОВАНИЕ И РЕГИСТРАЦИЯ МОДЕЛИ В MLFLOW

✅ Run ID: 71f3c92a962f48f4903846391a0a3c54
✅ Artifact URI: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/4/71f3c92a962f48f4903846391a0a3c54/artifacts

📊 Логирование метрик...
✅ Метрики залогированы

💾 Сохранение модели в MLflow...
  ✓ Модель сохранена: /var/folders/6g/s9qby9js2yd4gy8c0t6tftgh0000gp/T/tmpehn2yg6l/churn-classifier-autofeat/model.pkl

📤 Загрузка артефактов в MLflow...
✅ Модель успешно сохранена!
🏃 View run model_training_with_autofeat at: http://127.0.0.1:5001/#/experiments/4/runs/71f3c92a962f48f4903846391a0a3c54
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/4

🔗 View run: http://127.0.0.1:5001/#/experiments/4/runs/71f3c92a962f48f4903846391a0a3c54

⏳ Ожидание 3 секунды...

РЕГИСТРАЦИЯ МОДЕЛИ В MODEL REGISTRY

📝 Регистрация модели...
  Model URI: runs:/71f3c92a962f48f4903846391a0a3c54/churn-classifier-autofeat
  Model Name: churn-classifier-autofeat


Successfully registered model 'churn-classifier-autofeat'.
2025/12/07 16:37:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: churn-classifier-autofeat, version 1



✅ Модель зарегистрирована!
  Model Name: churn-classifier-autofeat
  Version: 1
  Run ID: 71f3c92a962f48f4903846391a0a3c54

📝 ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3:
model_version_id = 1
model_registred_name = "churn-classifier-autofeat"
run_id = "71f3c92a962f48f4903846391a0a3c54"


Created version '1' of model 'churn-classifier-autofeat'.


In [31]:
# Задание 3: Переменные для проверки
model_version_id = registered_model_version_id  # номер зарегистрированной модели
model_registred_name = registered_model_name  # название зарегистрированной модели
run_id = registered_run_id  # run_id эксперимента, в котором была зарегистрирована модель

print("="*70)
print("✅ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3")
print("="*70)
print(f"model_version_id = {model_version_id}")
print(f'model_registred_name = "{model_registred_name}"')
print(f'run_id = "{run_id}"')

✅ ЗНАЧЕНИЯ ДЛЯ ПРОВЕРКИ ЗАДАНИЯ 3
model_version_id = 1
model_registred_name = "churn-classifier-autofeat"
run_id = "71f3c92a962f48f4903846391a0a3c54"
